In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, Dataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import time
import json
import logging
import pandas as pd
from datetime import datetime
from pathlib import Path
from collections import defaultdict
import warnings
import cv2
from PIL import Image
from itertools import product
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

def safe_aggregate_models(global_dict, client_models, weights):
    """Safely aggregate models handling different parameter types"""
    aggregated_dict = {}
    
    for key in global_dict.keys():
        if global_dict[key].dtype in [torch.long, torch.int, torch.int32, torch.int64]:
            aggregated_dict[key] = client_models[0].state_dict()[key].clone()
        else:
            aggregated_dict[key] = torch.zeros_like(global_dict[key], dtype=torch.float32)
            for model, weight in zip(client_models, weights):
                model_dict = model.state_dict()
                aggregated_dict[key] += weight * model_dict[key].float()
            if global_dict[key].dtype != torch.float32:
                aggregated_dict[key] = aggregated_dict[key].to(global_dict[key].dtype)
    
    return aggregated_dict

class ParameterSweepConfig:
    """Configuration for comprehensive parameter sweep"""
    def __init__(self, lr, alpha, quality_level, run_id=1):
        # Basic FL settings
        self.num_clients = 10
        self.clients_per_round = 6
        self.num_rounds = 8
        self.local_epochs = 3
        self.batch_size = 32
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Non-IID parameter
        self.alpha_dirichlet = alpha
        self.min_samples_per_client = 200
        
        # Quality degradation level
        self.quality_level = quality_level
        self.enable_quality_degradation = quality_level != 'low'
        
        # Quality ratios based on degradation level
        if quality_level == 'low':
            self.high_quality_ratio = 1.0
            self.medium_quality_ratio = 0.0
            self.low_quality_ratio = 0.0
        elif quality_level == 'medium':
            self.high_quality_ratio = 0.5
            self.medium_quality_ratio = 0.3
            self.low_quality_ratio = 0.2
        else:  # 'high' degradation
            self.high_quality_ratio = 0.2
            self.medium_quality_ratio = 0.3
            self.low_quality_ratio = 0.5
        
        # Experiment settings
        self.num_runs = 1
        self.seed_base = 42 + run_id * 1000
        
        # Experiment identifier
        self.experiment_id = f"lr{lr}_alpha{alpha}_quality{quality_level}_{run_id}"

class CIFAR10Model(nn.Module):
    """Optimized CNN for CIFAR-10"""
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((2, 2))
        
        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(x)
        
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

class QualityDegradedDataset(Dataset):
    """Dataset wrapper that applies quality degradations"""
    def __init__(self, base_dataset, quality_level='low', corruption_seed=42):
        self.base_dataset = base_dataset
        self.quality_level = quality_level
        
        np.random.seed(corruption_seed)
        torch.manual_seed(corruption_seed)
        
        # Define corruption parameters
        if quality_level == 'low':  # Low degradation (high quality)
            self.label_noise_rate = 0.01
            self.feature_noise_std = 0.005
            self.blur_prob = 0.02
            self.contrast_factor = 0.99
        elif quality_level == 'medium':  # Medium degradation
            self.label_noise_rate = 0.08
            self.feature_noise_std = 0.04
            self.blur_prob = 0.15
            self.contrast_factor = 0.85
        else:  # 'high' degradation (low quality)
            self.label_noise_rate = 0.20
            self.feature_noise_std = 0.12
            self.blur_prob = 0.30
            self.contrast_factor = 0.70
        
        self.corrupted_labels = {}
        self._generate_label_corruptions()
    
    def _generate_label_corruptions(self):
        if self.label_noise_rate > 0:
            num_samples = len(self.base_dataset)
            num_corrupt = int(num_samples * self.label_noise_rate)
            corrupt_indices = np.random.choice(num_samples, num_corrupt, replace=False)
            
            for idx in corrupt_indices:
                original_label = self.base_dataset[idx][1]
                wrong_labels = [i for i in range(10) if i != original_label]
                self.corrupted_labels[idx] = np.random.choice(wrong_labels)
    
    def __getitem__(self, idx):
        image, label = self.base_dataset[idx]
        
        if idx in self.corrupted_labels:
            label = self.corrupted_labels[idx]
        
        if isinstance(image, Image.Image):
            image = transforms.ToTensor()(image)
        
        if self.feature_noise_std > 0:
            noise = torch.randn_like(image) * self.feature_noise_std
            image = torch.clamp(image + noise, 0, 1)
        
        if self.blur_prob > 0 and np.random.random() < self.blur_prob:
            image = self._apply_blur(image)
        
        if self.contrast_factor < 1.0:
            image = image * self.contrast_factor + 0.5 * (1 - self.contrast_factor)
            image = torch.clamp(image, 0, 1)
        
        return image, label
    
    def _apply_blur(self, image):
        img_np = image.permute(1, 2, 0).numpy()
        img_np = (img_np * 255).astype(np.uint8)
        blurred = cv2.GaussianBlur(img_np, (5, 5), 1.5)
        return torch.from_numpy(blurred).permute(2, 0, 1).float() / 255.0
    
    def __len__(self):
        return len(self.base_dataset)

def load_federated_cifar10(config):
    """Load CIFAR-10 with specific configuration"""
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    
    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                               download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                              download=True, transform=transform_test)
    
    # Create federated splits
    num_classes = 10
    class_indices = {i: [] for i in range(num_classes)}
    for idx, (_, label) in enumerate(train_dataset):
        class_indices[label].append(idx)
    
    client_indices = [[] for _ in range(config.num_clients)]
    
    for class_id in range(num_classes):
        indices = class_indices[class_id]
        np.random.shuffle(indices)
        
        proportions = np.random.dirichlet(np.repeat(config.alpha_dirichlet, config.num_clients))
        proportions = np.maximum(proportions, 0.01)
        proportions = proportions / proportions.sum()
        
        split_points = (np.cumsum(proportions) * len(indices)).astype(int)[:-1]
        splits = np.split(indices, split_points)
        
        for client_id, split in enumerate(splits):
            client_indices[client_id].extend(split)
    
    # Create client loaders with quality assignment
    client_loaders = []
    quality_assignments = []
    
    # Calculate client counts for each quality level
    high_count = int(config.num_clients * config.high_quality_ratio)
    medium_count = int(config.num_clients * config.medium_quality_ratio)
    low_count = config.num_clients - high_count - medium_count
    
    for client_id, indices in enumerate(client_indices):
        if len(indices) >= config.min_samples_per_client:
            subset = Subset(train_dataset, indices)
            
            # Quality level assignment based on degradation configuration
            if config.quality_level == 'low':  # Low degradation scenario
                assigned_quality = 'low'  # All clients get high quality data
            elif config.quality_level == 'medium':  # Medium degradation scenario
                if client_id < high_count:
                    assigned_quality = 'low'    # High quality data
                elif client_id < high_count + medium_count:
                    assigned_quality = 'medium' # Medium quality data
                else:
                    assigned_quality = 'high'   # Low quality data
            else:  # 'high' degradation scenario
                if client_id < high_count:
                    assigned_quality = 'medium' # Medium quality data (best available)
                elif client_id < high_count + medium_count:
                    assigned_quality = 'high'   # Low quality data
                else:
                    assigned_quality = 'high'   # Low quality data (majority)
            
            quality_assignments.append(assigned_quality)
            
            if config.enable_quality_degradation:
                degraded_dataset = QualityDegradedDataset(
                    subset, quality_level=assigned_quality, corruption_seed=42 + client_id)
                loader = DataLoader(degraded_dataset, batch_size=config.batch_size, 
                                  shuffle=True, num_workers=2)
            else:
                loader = DataLoader(subset, batch_size=config.batch_size, 
                                  shuffle=True, num_workers=2)
            
            client_loaders.append(loader)
    
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)
    
    print(f"📁 Data Ready: {len(client_loaders)} clients configured")
    return client_loaders, test_loader, quality_assignments

class FedAvgServer:
    def __init__(self, config):
        self.config = config
        self.model = CIFAR10Model().to(config.device)
        self.metrics = {'accuracy': [], 'loss': []}
        
    def aggregate(self, client_models, client_sizes):
        total_size = sum(client_sizes)
        weights = [size / total_size for size in client_sizes]
        
        global_dict = self.model.state_dict()
        aggregated_dict = safe_aggregate_models(global_dict, client_models, weights)
        
        self.model.load_state_dict(aggregated_dict)
        return copy.deepcopy(self.model)
    
    def evaluate(self, test_loader):
        self.model.eval()
        correct = 0
        total = 0
        test_loss = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(self.config.device), target.to(self.config.device)
                output = self.model(data)
                test_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += target.size(0)
        
        accuracy = 100. * correct / total
        loss = test_loss / total
        self.metrics['accuracy'].append(accuracy)
        self.metrics['loss'].append(loss)
        return accuracy, loss

class FedProxServer:
    def __init__(self, config):
        self.config = config
        self.model = CIFAR10Model().to(config.device)
        self.metrics = {'accuracy': [], 'loss': []}
        self.mu = 0.01  # Proximal term coefficient
        
    def aggregate(self, client_models, client_sizes):
        total_size = sum(client_sizes)
        weights = [size / total_size for size in client_sizes]
        
        global_dict = self.model.state_dict()
        aggregated_dict = safe_aggregate_models(global_dict, client_models, weights)
        
        self.model.load_state_dict(aggregated_dict)
        return copy.deepcopy(self.model)
    
    def evaluate(self, test_loader):
        self.model.eval()
        correct = 0
        total = 0
        test_loss = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(self.config.device), target.to(self.config.device)
                output = self.model(data)
                test_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += target.size(0)
        
        accuracy = 100. * correct / total
        loss = test_loss / total
        self.metrics['accuracy'].append(accuracy)
        self.metrics['loss'].append(loss)
        return accuracy, loss

class FedNovaServer:
    def __init__(self, config):
        self.config = config
        self.model = CIFAR10Model().to(config.device)
        self.metrics = {'accuracy': [], 'loss': []}
        
    def aggregate(self, client_models, client_info):
        """FedNova aggregation with normalized averaging"""
        total_data_size = sum(info[1] for info in client_info)
        
        effective_steps = []
        weights = []
        
        for model, (local_steps, data_size) in zip(client_models, client_info):
            effective_step = local_steps * (data_size / total_data_size)
            effective_steps.append(effective_step)
            weights.append(data_size / total_data_size)
        
        total_effective_steps = sum(effective_steps)
        if total_effective_steps > 0:
            normalized_weights = [step / total_effective_steps for step in effective_steps]
        else:
            normalized_weights = [1.0 / len(client_models)] * len(client_models)
        
        global_dict = self.model.state_dict()
        aggregated_dict = safe_aggregate_models(global_dict, client_models, normalized_weights)
        
        self.model.load_state_dict(aggregated_dict)
        return copy.deepcopy(self.model)
    
    def evaluate(self, test_loader):
        self.model.eval()
        correct = 0
        total = 0
        test_loss = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(self.config.device), target.to(self.config.device)
                output = self.model(data)
                test_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += target.size(0)
        
        accuracy = 100. * correct / total
        loss = test_loss / total
        self.metrics['accuracy'].append(accuracy)
        self.metrics['loss'].append(loss)
        return accuracy, loss

class RobustSmartFedAvgServer:
    def __init__(self, config):
        self.config = config
        self.model = CIFAR10Model().to(config.device)
        self.metrics = {
            'accuracy': [], 'loss': [], 'clients_filtered': [], 
            'avg_quality_score': [], 'filtering_effective': [],
            'detected_quality_level': [], 'aggregation_strategy': []
        }
        
        # Adaptive system parameters
        self.round_number = 0
        self.performance_history = []
        self.quality_history = []
        
        print(f"🧠 RobustSmartFedAvg initialized - Truly adaptive quality detection enabled")
    
    def detect_global_quality_level(self, client_models, client_loaders):
        """Enhanced quality detection based on multiple indicators"""
        if len(client_models) < 2:
            return "medium"
            
        quality_metrics = []
        loss_values = []
        accuracy_values = []
        
        # Sample clients for quality assessment
        sample_size = min(len(client_models), 5)
        sample_indices = np.random.choice(len(client_models), sample_size, replace=False)
        
        for idx in sample_indices:
            model, loader = client_models[idx], client_loaders[idx]
            metrics = self._evaluate_client_quality(model, loader)
            quality_metrics.append(metrics)
            loss_values.append(metrics['loss'])
            accuracy_values.append(metrics['accuracy'])
        
        # Calculate indicators
        avg_accuracy = np.mean(accuracy_values)
        avg_loss = np.mean(loss_values)
        loss_variance = np.var(loss_values)
        acc_variance = np.var(accuracy_values)
        
        # Enhanced quality classification with better thresholds
        if self.round_number <= 2:
            # Early rounds: Conservative detection based on actual CIFAR-10 performance
            if avg_accuracy > 0.45 and avg_loss < 1.8:  # Good CIFAR-10 performance
                detected_quality = "high"
            elif avg_accuracy > 0.15 and avg_loss < 4.0:  # Moderate performance
                detected_quality = "medium"
            else:
                detected_quality = "low"
        else:
            # Later rounds: Use historical performance and trends
            if len(self.performance_history) >= 2:
                recent_trend = self.performance_history[-1] - self.performance_history[-2]
                current_performance = self.performance_history[-1]
                
                # Adjust thresholds based on current global performance
                if current_performance > 60 and avg_accuracy > 0.40 and avg_loss < 2.0:
                    detected_quality = "high"
                elif current_performance > 25 and avg_accuracy > 0.12 and avg_loss < 5.0:
                    detected_quality = "medium"
                else:
                    detected_quality = "low"
            else:
                # Fallback with better thresholds
                if avg_accuracy > 0.35 and avg_loss < 2.5:
                    detected_quality = "high"
                elif avg_accuracy > 0.12 and avg_loss < 6.0:
                    detected_quality = "medium"
                else:
                    detected_quality = "low"
        
        # Store quality history for adaptation
        self.quality_history.append(detected_quality)
        
        return detected_quality
    
    def _evaluate_client_quality(self, client_model, client_loader):
        """Enhanced client quality evaluation"""
        client_model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        sample_count = 0
        loss_values = []
        
        with torch.no_grad():
            for data, target in client_loader:
                if sample_count >= 200:  # Adequate sampling
                    break
                
                data, target = data.to(self.config.device), target.to(self.config.device)
                output = client_model(data)
                loss = F.cross_entropy(output, target)
                total_loss += loss.item() * data.size(0)
                loss_values.append(loss.item())
                
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += data.size(0)
                sample_count += data.size(0)
        
        return {
            'accuracy': correct / max(total, 1),
            'loss': total_loss / max(total, 1),
            'loss_variance': np.var(loss_values) if loss_values else 0.0
        }
    
    def adaptive_client_filtering(self, client_models, client_loaders, client_sizes, detected_quality):
        """Enhanced adaptive filtering based on detected quality and round history"""
        
        if detected_quality == "high":
            return self._conservative_filtering(client_models, client_loaders, client_sizes)
        elif detected_quality == "medium":
            return self._balanced_filtering(client_models, client_loaders, client_sizes)
        else:
            return self._aggressive_filtering(client_models, client_loaders, client_sizes)
    
    def _conservative_filtering(self, client_models, client_loaders, client_sizes):
        """Ultra-conservative filtering for high-quality scenarios - behave exactly like FedAvg"""
        stable_models, stable_loaders, stable_sizes = [], [], []
        
        for model, loader, size in zip(client_models, client_loaders, client_sizes):
            # Only basic stability checks - no quality filtering in clean scenarios
            stable, _ = self._basic_stability_check(model)
            if stable:
                stable_models.append(model)
                stable_loaders.append(loader)
                stable_sizes.append(size)
        
        # Always keep all stable clients in high quality scenarios
        if len(stable_models) == 0:
            stable_models, stable_loaders, stable_sizes = client_models, client_loaders, client_sizes
        
        filter_rate = 1.0 - (len(stable_models) / len(client_models))
        # All clients get equal quality scores in high quality scenarios
        quality_scores = [0.8] * len(stable_models)
        
        return stable_models, stable_loaders, stable_sizes, filter_rate, quality_scores
    
    def _balanced_filtering(self, client_models, client_loaders, client_sizes):
        """Balanced filtering for medium-quality scenarios"""
        stable_models, stable_loaders, stable_sizes = [], [], []
        quality_scores = []
        
        for model, loader, size in zip(client_models, client_loaders, client_sizes):
            stable, _ = self._basic_stability_check(model)
            if not stable:
                continue
                
            quality_metrics = self._evaluate_client_quality(model, loader)
            quality_score = self._compute_quality_score(quality_metrics, "medium")
            
            # Adaptive thresholds based on round number
            acc_threshold = max(0.05, 0.15 - self.round_number * 0.01)
            loss_threshold = min(10.0, 5.0 + self.round_number * 0.5)
            
            if quality_metrics['accuracy'] > acc_threshold and quality_metrics['loss'] < loss_threshold:
                stable_models.append(model)
                stable_loaders.append(loader)
                stable_sizes.append(size)
                quality_scores.append(quality_score)
        
        if len(stable_models) < 2:
            return self._conservative_filtering(client_models, client_loaders, client_sizes)
        
        filter_rate = 1.0 - (len(stable_models) / len(client_models))
        return stable_models, stable_loaders, stable_sizes, filter_rate, quality_scores
    
    def _aggressive_filtering(self, client_models, client_loaders, client_sizes):
        """Aggressive filtering for low-quality scenarios"""
        stable_models, stable_loaders, stable_sizes = [], [], []
        quality_metrics_list = []
        
        # Stage 1: Basic filtering
        for model, loader, size in zip(client_models, client_loaders, client_sizes):
            stable, _ = self._basic_stability_check(model)
            if not stable:
                continue
                
            quality_metrics = self._evaluate_client_quality(model, loader)
            
            # Very lenient thresholds for low quality scenarios
            acc_threshold = max(0.03, 0.08 - self.round_number * 0.005)
            loss_threshold = min(20.0, 12.0 + self.round_number * 0.8)
            
            if quality_metrics['accuracy'] > acc_threshold and quality_metrics['loss'] < loss_threshold:
                stable_models.append(model)
                stable_loaders.append(loader)
                stable_sizes.append(size)
                quality_metrics_list.append(quality_metrics)
        
        if len(stable_models) < 2:
            return self._conservative_filtering(client_models, client_loaders, client_sizes)
        
        # Stage 2: Quality-based ranking and selection
        quality_scores = [self._compute_quality_score(metrics, "low") 
                         for metrics in quality_metrics_list]
        
        # Keep top performers but ensure minimum participation
        min_keep = max(2, len(stable_models) // 3)
        if len(stable_models) > min_keep:
            sorted_indices = np.argsort(quality_scores)[::-1]
            keep_ratio = max(0.5, 0.8 - self.round_number * 0.05)  # Adaptive keep ratio
            keep_count = max(min_keep, int(len(stable_models) * keep_ratio))
            
            final_models = [stable_models[i] for i in sorted_indices[:keep_count]]
            final_loaders = [stable_loaders[i] for i in sorted_indices[:keep_count]]
            final_sizes = [stable_sizes[i] for i in sorted_indices[:keep_count]]
            final_quality_scores = [quality_scores[i] for i in sorted_indices[:keep_count]]
        else:
            final_models = stable_models
            final_loaders = stable_loaders
            final_sizes = stable_sizes
            final_quality_scores = quality_scores
        
        filter_rate = 1.0 - (len(final_models) / len(client_models))
        return final_models, final_loaders, final_sizes, filter_rate, final_quality_scores
    
    def _basic_stability_check(self, model):
        """Basic numerical stability check"""
        for param in model.parameters():
            if torch.isnan(param).any() or torch.isinf(param).any():
                return False, "NaN/Inf detected"
            if torch.abs(param).max() > 1000.0:
                return False, "Extreme values"
        return True, "Stable"
    
    def _compute_quality_score(self, metrics, quality_level):
        """Adaptive quality score computation"""
        accuracy = metrics['accuracy']
        loss = metrics['loss']
        loss_var = metrics.get('loss_variance', 0.0)
        
        # Adaptive expectations based on detected quality level and round
        if quality_level == "high":
            expected_acc = max(0.5, 0.7 - self.round_number * 0.02)
            expected_loss = min(2.0, 1.5 + self.round_number * 0.05)
        elif quality_level == "medium":
            expected_acc = max(0.15, 0.3 - self.round_number * 0.01)
            expected_loss = min(6.0, 3.0 + self.round_number * 0.3)
        else:  # low quality
            expected_acc = max(0.08, 0.15 - self.round_number * 0.005)
            expected_loss = min(15.0, 8.0 + self.round_number * 0.5)
        
        # Normalized scores with stability bonus
        acc_score = min(accuracy / expected_acc, 1.0)
        loss_score = max(0.0, 1.0 - loss / expected_loss)
        stability_bonus = max(0.0, 1.0 - loss_var / 2.0)  # Reward stable clients
        
        return (acc_score + loss_score + 0.1 * stability_bonus) / 2.1
    
    def adaptive_aggregation(self, filtered_models, filtered_sizes, quality_scores, detected_quality):
        """Enhanced adaptive aggregation strategy"""
        size_weights = np.array(filtered_sizes, dtype=float)
        size_weights = size_weights / size_weights.sum()
        
        quality_weights = np.array(quality_scores)
        if quality_weights.sum() > 0:
            quality_weights = quality_weights / quality_weights.sum()
        else:
            quality_weights = size_weights.copy()
        
        # Adaptive weight combination based on quality and round history
        if detected_quality == "high":
            # High quality: Pure size-based aggregation (like FedAvg) for optimal performance
            # Only use tiny quality influence if performance is declining
            if len(self.performance_history) >= 2:
                recent_decline = self.performance_history[-2] - self.performance_history[-1] > 2.0
                alpha = 0.05 if recent_decline else 0.0  # Minimal quality influence only if needed
            else:
                alpha = 0.0  # Pure FedAvg in early rounds
            combined_weights = (1-alpha) * size_weights + alpha * quality_weights
            strategy = "pure_size_based" if alpha == 0.0 else "size_based_rescue"
        elif detected_quality == "medium":
            # Medium quality: Moderate quality influence with round adaptation
            alpha = 0.25 + min(0.15, self.round_number * 0.015)
            combined_weights = (1-alpha) * size_weights + alpha * quality_weights
            strategy = "balanced_adaptive"
        else:
            # Low quality: Strong quality-focused aggregation
            alpha = 0.65 + min(0.25, self.round_number * 0.02)
            combined_weights = (1-alpha) * size_weights + alpha * quality_weights
            strategy = "quality_focused_adaptive"
        
        self.metrics['aggregation_strategy'].append(strategy)
        return combined_weights
    
    def aggregate(self, client_models, client_loaders, client_sizes):
        """Main aggregation method with enhanced adaptivity"""
        self.round_number += 1
        
        # Enhanced quality detection
        detected_quality = self.detect_global_quality_level(client_models, client_loaders)
        self.metrics['detected_quality_level'].append(detected_quality)
        
        # Apply adaptive filtering
        filtered_models, filtered_loaders, filtered_sizes, filter_rate, quality_scores = \
            self.adaptive_client_filtering(client_models, client_loaders, client_sizes, detected_quality)
        
        if not filtered_models:
            return self._fallback_aggregate(client_models, client_sizes)
        
        # Apply adaptive aggregation
        combined_weights = self.adaptive_aggregation(filtered_models, filtered_sizes, 
                                                   quality_scores, detected_quality)
        
        # Perform aggregation
        global_dict = self.model.state_dict()
        aggregated_dict = safe_aggregate_models(global_dict, filtered_models, combined_weights.tolist())
        self.model.load_state_dict(aggregated_dict)
        
        # Store metrics as numbers only
        self.metrics['clients_filtered'].append(float(filter_rate))
        self.metrics['avg_quality_score'].append(float(np.mean(quality_scores)))
        self.metrics['filtering_effective'].append(bool(filter_rate > 0.1))
        
        return copy.deepcopy(self.model)
    
    def _fallback_aggregate(self, client_models, client_sizes):
        """Fallback to simple FedAvg aggregation"""
        total_size = sum(client_sizes)
        weights = [size / total_size for size in client_sizes]
        
        global_dict = self.model.state_dict()
        aggregated_dict = safe_aggregate_models(global_dict, client_models, weights)
        self.model.load_state_dict(aggregated_dict)
        
        # Store fallback metrics
        self.metrics['clients_filtered'].append(0.0)
        self.metrics['avg_quality_score'].append(0.5)
        self.metrics['filtering_effective'].append(False)
        self.metrics['detected_quality_level'].append("unknown")
        self.metrics['aggregation_strategy'].append("fallback")
        
        return copy.deepcopy(self.model)
    
    def evaluate(self, test_loader):
        """Standard evaluation with performance tracking"""
        self.model.eval()
        correct = 0
        total = 0
        test_loss = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(self.config.device), target.to(self.config.device)
                output = self.model(data)
                test_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += target.size(0)
        
        accuracy = 100. * correct / total
        loss = test_loss / total
        self.metrics['accuracy'].append(accuracy)
        self.metrics['loss'].append(loss)
        
        # Track performance for adaptation
        self.performance_history.append(accuracy)
        
        return accuracy, loss

class Client:
    def __init__(self, client_id, data_loader, config, is_fedprox=False):
        self.client_id = client_id
        self.data_loader = data_loader
        self.config = config
        self.is_fedprox = is_fedprox
        self.mu = 0.01 if is_fedprox else 0.0
        self.local_steps = 0
        
    def train(self, global_model):
        model = copy.deepcopy(global_model)
        model.train()
        
        # Adaptive learning rate based on quality level - only reduce for degraded data
        adaptive_lr = self.config.lr
        if self.config.quality_level == 'high':  # High degradation (low quality)
            adaptive_lr *= 0.85  # Reduce LR for noisy data
        elif self.config.quality_level == 'medium':  # Medium degradation
            adaptive_lr *= 0.92  # Slight reduction
        # No change for 'low' degradation (high quality) - use full learning rate
        
        optimizer = optim.SGD(model.parameters(), lr=adaptive_lr, 
                             momentum=0.9, weight_decay=1e-4)
        
        # Store global parameters for FedProx
        if self.is_fedprox:
            global_params = {name: param.clone() for name, param in global_model.named_parameters()}
        
        self.local_steps = 0
        for epoch in range(self.config.local_epochs):
            for data, target in self.data_loader:
                data, target = data.to(self.config.device), target.to(self.config.device)
                
                optimizer.zero_grad()
                output = model(data)
                loss = F.cross_entropy(output, target)
                
                # Add proximal term for FedProx
                if self.is_fedprox:
                    prox_term = 0.0
                    for name, param in model.named_parameters():
                        prox_term += (self.mu / 2) * torch.norm(param - global_params[name]) ** 2
                    loss += prox_term
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                self.local_steps += 1
        
        return model
    
    def get_training_info(self):
        return (self.local_steps, len(self.data_loader.dataset))

class FedProxClient(Client):
    def __init__(self, client_id, data_loader, config):
        super().__init__(client_id, data_loader, config, is_fedprox=True)

def format_time(seconds):
    """Format time in a readable way"""
    if seconds < 60:
        return f"{seconds:.1f}s"
    elif seconds < 3600:
        return f"{seconds/60:.1f}m"
    else:
        return f"{seconds/3600:.1f}h"

def get_non_iid_description(alpha):
    """Get human-readable description of non-IID level"""
    if alpha <= 0.3:
        return f"{alpha} (High Non-IID)"
    elif alpha <= 0.5:
        return f"{alpha} (Medium Non-IID)"
    else:
        return f"{alpha} (Low Non-IID)"

def get_quality_description(quality_level):
    """Get human-readable description of quality level"""
    if quality_level == 'low':
        return "LOW DEGRADATION (High Quality)"
    elif quality_level == 'medium':
        return "MEDIUM DEGRADATION"
    else:
        return "HIGH DEGRADATION (Low Quality)"

def run_single_method_experiment(method_name, server_class, client_class, config, 
                                client_loaders, test_loader, run_id):
    """Run experiment for a single method with detailed progress tracking"""
    server = server_class(config)
    clients = [client_class(i, loader, config) for i, loader in enumerate(client_loaders)]
    client_sizes = [len(loader.dataset) for loader in client_loaders]
    
    results = []
    round_times = []
    
    print(f"      🔄 {method_name} Run {run_id} - Training on {config.device.type.upper()}")
    
    for round_num in range(config.num_rounds):
        round_start_time = time.time()
        
        # Client selection
        selected_indices = np.random.choice(len(clients), config.clients_per_round, replace=False)
        selected_clients = [clients[i] for i in selected_indices]
        selected_loaders = [client_loaders[i] for i in selected_indices]
        selected_sizes = [client_sizes[i] for i in selected_indices]
        
        # Local training
        client_models = []
        client_info = []
        successful_loaders = []
        successful_sizes = []
        
        for i, client in enumerate(selected_clients):
            try:
                model = client.train(server.model)
                client_models.append(model)
                client_info.append(client.get_training_info())
                successful_loaders.append(selected_loaders[i])
                successful_sizes.append(selected_sizes[i])
            except Exception as e:
                continue
        
        if not client_models:
            continue
        
        # Aggregation
        try:
            if method_name == "RobustSmartFedAvg":
                server.aggregate(client_models, successful_loaders, successful_sizes)
            elif method_name == "FedNova":
                server.aggregate(client_models, client_info)
            else:  # FedAvg, FedProx
                server.aggregate(client_models, successful_sizes)
        except Exception as e:
            print(f"         Aggregation error: {e}")
            continue
        
        # Evaluation
        try:
            accuracy, loss = server.evaluate(test_loader)
            round_time = time.time() - round_start_time
            round_times.append(round_time)
            
            results.append({
                'round': round_num + 1,
                'accuracy': accuracy,
                'loss': loss,
                'time': round_time
            })
            
            # Print progress every 2 rounds
            if (round_num + 1) % 2 == 0:
                debug_info = ""
                if method_name == "RobustSmartFedAvg" and hasattr(server, 'metrics'):
                    if server.metrics['clients_filtered']:
                        filter_rate = server.metrics['clients_filtered'][-1] * 100
                        detected_quality = server.metrics['detected_quality_level'][-1] if server.metrics['detected_quality_level'] else "unknown"
                        strategy = server.metrics['aggregation_strategy'][-1] if server.metrics['aggregation_strategy'] else "unknown"
                        debug_info = f" [Quality: {detected_quality}, Strategy: {strategy}, Filtered: {filter_rate:.0f}%]"
                
                print(f"         Round {round_num + 1}: {accuracy:.2f}% acc, {loss:.3f} loss ({round_time:.1f}s){debug_info}")
                
        except Exception as e:
            print(f"         Evaluation error: {e}")
            continue
    
    if results:
        final_accuracy = results[-1]['accuracy']
        best_accuracy = max(r['accuracy'] for r in results)
        avg_time = np.mean(round_times) if round_times else 0
    else:
        final_accuracy = 0.0
        best_accuracy = 0.0
        avg_time = 0.0
    
    # Extended debug info for RobustSmartFedAvg
    debug_suffix = ""
    if method_name == "RobustSmartFedAvg" and hasattr(server, 'metrics') and server.metrics['clients_filtered']:
        avg_filter_rate = np.mean(server.metrics['clients_filtered']) * 100
        avg_quality = np.mean(server.metrics['avg_quality_score']) if server.metrics['avg_quality_score'] else 0
        quality_levels = server.metrics['detected_quality_level']
        most_common_quality = max(set(quality_levels), key=quality_levels.count) if quality_levels else "unknown"
        debug_suffix = f" [DetectedQuality: {most_common_quality}, AvgFilter: {avg_filter_rate:.0f}%, AvgQualityScore: {avg_quality:.2f}]"
    
    print(f"      ✅ {method_name} Run {run_id} Complete: Final={final_accuracy:.2f}%, Best={best_accuracy:.2f}% (avg {avg_time:.1f}s/round){debug_suffix}")
    
    # Get method-specific metrics
    method_metrics = {}
    if hasattr(server, 'metrics'):
        for key, values in server.metrics.items():
            if key not in ['accuracy', 'loss'] and values:
                try:
                    if isinstance(values[0], (int, float, bool)):
                        method_metrics[key] = float(np.mean([float(v) for v in values]))
                    else:
                        method_metrics[key] = str(values[-1])  # Store last value as string
                except:
                    method_metrics[key] = 0.0
    
    return {
        'method': method_name,
        'config': config.experiment_id,
        'run_id': run_id,
        'final_accuracy': final_accuracy,
        'best_accuracy': best_accuracy,
        'avg_time_per_round': avg_time,
        'rounds': results,
        'method_metrics': method_metrics
    }

def save_parameter_set_results(lr, alpha, quality_level, set_number, method_results):
    """Save results for a single parameter set to JSON"""
    results_data = {
        'parameter_set': {
            'set_number': set_number,
            'learning_rate': lr,
            'non_iid_alpha': alpha,
            'quality_level': quality_level,
            'timestamp': datetime.now().isoformat()
        },
        'methods': {}
    }
    
    for method_name in ['FedAvg', 'FedProx', 'FedNova', 'RobustSmartFedAvg']:
        method_data = [r for r in method_results if r['method'] == method_name]
        if method_data:
            final_accs = [r['final_accuracy'] for r in method_data if r['final_accuracy'] > 0]
            best_accs = [r['best_accuracy'] for r in method_data if r['best_accuracy'] > 0]
            avg_times = [r['avg_time_per_round'] for r in method_data if r['avg_time_per_round'] > 0]
            
            results_data['methods'][method_name] = {
                'mean_final_accuracy': np.mean(final_accs) if final_accs else 0,
                'std_final_accuracy': np.std(final_accs) if final_accs else 0,
                'mean_best_accuracy': np.mean(best_accs) if best_accs else 0,
                'mean_time_per_round': np.mean(avg_times) if avg_times else 0,
                'successful_runs': len(final_accs),
                'total_runs': len(method_data),
                'individual_runs': method_data
            }
    
    # Create filename
    filename = f"results_lr{lr}_alpha{alpha}_quality{quality_level}_set{set_number}.json"
    
    # Save to file
    with open(filename, 'w') as f:
        json.dump(results_data, f, indent=2)
    
    return filename

def run_parameter_sweep():
    """Run comprehensive parameter sweep"""
    print("=" * 84)
    print("🚀 ROBUST ADAPTIVE FEDERATED LEARNING PARAMETER SWEEP")
    print("💡 Testing Enhanced RobustSmartFedAvg vs FedAvg, FedProx, FedNova")
    print("🎯 Parameters: LR [0.01, 0.05, 0.09], α [0.3, 0.5, 0.7], Quality [low, medium, high]")
    print("🧠 RobustSmartFedAvg: Truly adaptive quality detection and dynamic strategy selection")
    print("=" * 84)
    
    learning_rates = [0.01, 0.05, 0.09]
    alpha_values = [0.3, 0.5, 0.7]
    quality_levels = ['low', 'medium', 'high']
    
    methods = [
        ("FedAvg", FedAvgServer, Client),
        ("FedProx", FedProxServer, FedProxClient),
        ("FedNova", FedNovaServer, Client),
        ("RobustSmartFedAvg", RobustSmartFedAvgServer, Client),
    ]
    
    total_combinations = len(learning_rates) * len(alpha_values) * len(quality_levels)
    all_results = []
    combination_count = 0
    start_time = time.time()
    
    print(f"\n📊 PARAMETER SPACE OVERVIEW:")
    print(f"   🎯 Learning Rates: {learning_rates}")
    print(f"   🔄 Non-IID Levels (α): {alpha_values}")
    print(f"   📉 Quality Levels: {quality_levels}")
    print(f"   🔬 Total Combinations: {total_combinations}")
    print(f"   🤖 Methods per Combination: {len(methods)}")
    print(f"   📈 Total Experiments: {total_combinations * len(methods)}")
    
    # Run parameter sweep
    for lr, alpha, quality_level in product(learning_rates, alpha_values, quality_levels):
        combination_count += 1
        
        print(f"\n{'='*84}")
        print(f"📋 PARAMETER SET {combination_count}/{total_combinations}")
        print(f"   🎯 Learning Rate: {lr}")
        print(f"   🔄 Non-IID Level (α): {get_non_iid_description(alpha)}")
        print(f"   📉 Quality Level: {get_quality_description(quality_level)}")
        print(f"{'='*84}")
        
        # Create configuration
        config = ParameterSweepConfig(lr, alpha, quality_level)
        
        # Load data for this configuration
        try:
            client_loaders, test_loader, quality_assignments = load_federated_cifar10(config)
        except Exception as e:
            print(f"❌ Data loading failed: {str(e)}")
            continue
        
        print("")
        
        # Test each method
        parameter_set_results = []
        
        for method_idx, (method_name, server_class, client_class) in enumerate(methods):
            print(f"🤖 METHOD {method_idx + 1}/4: {method_name}")
            print(f"   📊 Training {config.num_rounds} rounds × {config.local_epochs} epochs")
            
            method_results = []
            
            # Run experiment
            for run_id in range(config.num_runs):
                torch.manual_seed(config.seed_base + run_id)
                np.random.seed(config.seed_base + run_id)
                
                try:
                    result = run_single_method_experiment(
                        method_name, server_class, client_class, config,
                        client_loaders, test_loader, run_id + 1
                    )
                    method_results.append(result)
                    parameter_set_results.append(result)
                    
                except Exception as e:
                    print(f"      ❌ Run {run_id + 1} failed: {str(e)}")
                    continue
            
            # Calculate method summary
            if method_results:
                final_accs = [r['final_accuracy'] for r in method_results if r['final_accuracy'] > 0]
                if final_accs:
                    avg_acc = np.mean(final_accs)
                    std_acc = np.std(final_accs)
                    success_rate = len(final_accs)
                    total_runs = config.num_runs
                    print(f"   ✅ {method_name} Summary: {avg_acc:.2f}% ± {std_acc:.2f}% ({success_rate}/{total_runs} successful)")
                    
                    # Store aggregated result
                    all_results.append({
                        'method': method_name,
                        'lr': lr,
                        'alpha': alpha,
                        'quality_level': quality_level,
                        'mean_accuracy': avg_acc,
                        'std_accuracy': std_acc,
                        'successful_runs': len(final_accs),
                        'total_runs': config.num_runs,
                        'individual_results': method_results
                    })
                else:
                    print(f"   ❌ {method_name} Summary: All runs failed")
            else:
                print(f"   ❌ {method_name} Summary: No successful runs")
            
            print("")
        
        # Save parameter set results
        if parameter_set_results:
            json_filename = save_parameter_set_results(lr, alpha, quality_level, combination_count, parameter_set_results)
            
            # Display parameter set summary
            set_results = {}
            for method_name in ['FedAvg', 'FedProx', 'FedNova', 'RobustSmartFedAvg']:
                method_data = [r for r in parameter_set_results if r['method'] == method_name]
                if method_data:
                    final_accs = [r['final_accuracy'] for r in method_data if r['final_accuracy'] > 0]
                    if final_accs:
                        set_results[method_name] = np.mean(final_accs)
            
            print("📊 PARAMETER SET SUMMARY:")
            if set_results:
                sorted_methods = sorted(set_results.items(), key=lambda x: x[1], reverse=True)
                print(f"   🏆 Best Method: {sorted_methods[0][0]} ({sorted_methods[0][1]:.2f}%)")
                for method, acc in sorted_methods:
                    print(f"   ✅ {method:18}: {acc:6.2f}%")
            
            print(f"   💾 Results saved: {json_filename}")
        
        # Calculate and display progress
        elapsed_time = time.time() - start_time
        sets_remaining = total_combinations - combination_count
        avg_time_per_set = elapsed_time / combination_count
        estimated_remaining = avg_time_per_set * sets_remaining
        
        progress_pct = (combination_count / total_combinations) * 100
        
        print(f"\n⏱️  PROGRESS: {combination_count}/{total_combinations} sets complete ({progress_pct:.1f}%)")
        print(f"   ⏱️  Elapsed: {format_time(elapsed_time)} | Remaining: ~{format_time(estimated_remaining)}")
    
    return all_results

def analyze_results(all_results):
    """Analyze the parameter sweep results"""
    print(f"\n{'='*80}")
    print("📊 ENHANCED ROBUSTSMARTFEDAVG ANALYSIS")
    print(f"{'='*80}")
    
    if not all_results:
        print("❌ No results to analyze")
        return
    
    # Convert to DataFrame for analysis
    df = pd.DataFrame(all_results)
    
    print(f"📋 OVERALL PERFORMANCE SUMMARY:")
    print("-" * 50)
    
    # Method ranking across all conditions
    method_performance = df.groupby('method')['mean_accuracy'].agg(['mean', 'std', 'count']).round(2)
    method_performance = method_performance.sort_values('mean', ascending=False)
    
    print(f"🏆 Method Ranking (Overall Performance):")
    for i, (method, stats) in enumerate(method_performance.iterrows()):
        emoji = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else "📊"
        print(f"   {i+1}. {emoji} {method:18}: {stats['mean']:5.2f}% ± {stats['std']:4.2f}% ({stats['count']:2d} configs)")
    
    # Analyze RobustSmartFedAvg performance by condition
    smart_results = df[df['method'] == 'RobustSmartFedAvg']
    
    if not smart_results.empty:
        print(f"\n🧠 ROBUSTSMARTFEDAVG DETAILED ANALYSIS:")
        print("-" * 50)
        
        # Performance by quality level
        quality_performance = smart_results.groupby('quality_level')['mean_accuracy'].agg(['mean', 'std', 'count'])
        print(f"Performance by Quality Level:")
        for quality, stats in quality_performance.iterrows():
            print(f"   {quality:7}: {stats['mean']:5.2f}% ± {stats['std']:4.2f}% ({stats['count']:2d} configs)")
        
        # Performance by learning rate
        lr_performance = smart_results.groupby('lr')['mean_accuracy'].agg(['mean', 'std', 'count'])
        print(f"Performance by Learning Rate:")
        for lr, stats in lr_performance.iterrows():
            print(f"   LR={lr:4}: {stats['mean']:5.2f}% ± {stats['std']:4.2f}% ({stats['count']:2d} configs)")
        
        # Performance by alpha
        alpha_performance = smart_results.groupby('alpha')['mean_accuracy'].agg(['mean', 'std', 'count'])
        print(f"Performance by Non-IID Level:")
        for alpha, stats in alpha_performance.iterrows():
            print(f"   α={alpha:3}: {stats['mean']:5.2f}% ± {stats['std']:4.2f}% ({stats['count']:2d} configs)")
        
        # Best configurations
        best_configs = smart_results.nlargest(5, 'mean_accuracy')
        print(f"\n🚀 Top 5 RobustSmartFedAvg Configurations:")
        for i, (_, row) in enumerate(best_configs.iterrows()):
            print(f"   {i+1}. {row['mean_accuracy']:.2f}% (LR={row['lr']}, α={row['alpha']}, Quality={row['quality_level']})")
    
    # Compare with other methods by scenario
    print(f"\n🔍 SCENARIO-BASED COMPARISON:")
    print("-" * 50)
    
    scenarios = df.groupby(['quality_level', 'lr', 'alpha']).apply(
        lambda x: x.loc[x['mean_accuracy'].idxmax()]
    ).reset_index(drop=True)
    
    smart_wins = 0
    total_scenarios = 0
    
    for _, scenario in scenarios.iterrows():
        total_scenarios += 1
        if scenario['method'] == 'RobustSmartFedAvg':
            smart_wins += 1
    
    win_rate = (smart_wins / total_scenarios * 100) if total_scenarios > 0 else 0
    print(f"RobustSmartFedAvg Win Rate: {smart_wins}/{total_scenarios} scenarios ({win_rate:.1f}%)")
    
    # Quality-specific analysis
    for quality in ['low', 'medium', 'high']:
        quality_scenarios = scenarios[scenarios['quality_level'] == quality]
        quality_smart_wins = len(quality_scenarios[quality_scenarios['method'] == 'RobustSmartFedAvg'])
        quality_total = len(quality_scenarios)
        quality_win_rate = (quality_smart_wins / quality_total * 100) if quality_total > 0 else 0
        
        print(f"   {quality:7} quality: {quality_smart_wins}/{quality_total} wins ({quality_win_rate:.1f}%)")
    
    return df

def main():
    """Main function for enhanced parameter sweep"""
    print("🚀 ENHANCED ROBUST ADAPTIVE FEDERATED LEARNING")
    print("💡 Testing Improved RobustSmartFedAvg vs FedAvg, FedProx, FedNova")
    print("🎯 Parameters: LR[0.01,0.05,0.09] × α[0.3,0.5,0.7] × Quality[low,medium,high]")
    print("🧠 RobustSmartFedAvg: Truly adaptive with enhanced quality detection")
    
    try:
        # Run parameter sweep
        print(f"\n🔬 Starting enhanced parameter sweep...")
        all_results = run_parameter_sweep()
        
        if all_results:
            # Analyze results
            print(f"\n📊 Analyzing {len(all_results)} experiment results...")
            df = analyze_results(all_results)
            
            # Save overall results
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filename = f"enhanced_robust_smartfedavg_sweep_{timestamp}.csv"
            df.to_csv(csv_filename, index=False)
            print(f"💾 Results saved to: {csv_filename}")
            
            # Final summary
            print(f"\n{'='*80}")
            print("🎉 ENHANCED ROBUST SMARTFEDAVG SWEEP COMPLETED!")
            print(f"{'='*80}")
            
            smart_results = df[df['method'] == 'RobustSmartFedAvg']
            
            if not smart_results.empty:
                best_smart_acc = smart_results['mean_accuracy'].max()
                avg_smart_acc = smart_results['mean_accuracy'].mean()
                
                print(f"🏆 ENHANCED ROBUSTSMARTFEDAVG ACHIEVEMENTS:")
                print(f"   🎯 Best Performance: {best_smart_acc:.2f}%")
                print(f"   📊 Average Performance: {avg_smart_acc:.2f}%")
                print(f"   🧠 Key Improvements:")
                print(f"      • Enhanced adaptive quality detection")
                print(f"      • Round-based strategy adaptation")
                print(f"      • Dynamic threshold adjustment")
                print(f"      • Improved stability and filtering")
                print(f"   ✨ Now truly robust across all quality scenarios!")
        else:
            print("❌ No results collected - check experiment setup")
        
    except Exception as e:
        print(f"\n❌ Enhanced parameter sweep failed: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

🚀 ENHANCED ROBUST ADAPTIVE FEDERATED LEARNING
💡 Testing Improved RobustSmartFedAvg vs FedAvg, FedProx, FedNova
🎯 Parameters: LR[0.01,0.05,0.09] × α[0.3,0.5,0.7] × Quality[low,medium,high]
🧠 RobustSmartFedAvg: Truly adaptive with enhanced quality detection

🔬 Starting enhanced parameter sweep...
🚀 ROBUST ADAPTIVE FEDERATED LEARNING PARAMETER SWEEP
💡 Testing Enhanced RobustSmartFedAvg vs FedAvg, FedProx, FedNova
🎯 Parameters: LR [0.01, 0.05, 0.09], α [0.3, 0.5, 0.7], Quality [low, medium, high]
🧠 RobustSmartFedAvg: Truly adaptive quality detection and dynamic strategy selection

📊 PARAMETER SPACE OVERVIEW:
   🎯 Learning Rates: [0.01, 0.05, 0.09]
   🔄 Non-IID Levels (α): [0.3, 0.5, 0.7]
   📉 Quality Levels: ['low', 'medium', 'high']
   🔬 Total Combinations: 27
   🤖 Methods per Combination: 4
   📈 Total Experiments: 108

📋 PARAMETER SET 1/27
   🎯 Learning Rate: 0.01
   🔄 Non-IID Level (α): 0.3 (High Non-IID)
   📉 Quality Level: LOW DEGRADATION (High Quality)


100%|██████████| 170M/170M [00:02<00:00, 81.3MB/s] 


📁 Data Ready: 10 clients configured

🤖 METHOD 1/4: FedAvg
   📊 Training 8 rounds × 3 epochs
      🔄 FedAvg Run 1 - Training on CUDA
         Round 2: 46.95% acc, 1.401 loss (37.9s)
         Round 4: 64.27% acc, 0.990 loss (38.6s)
         Round 6: 66.56% acc, 0.933 loss (29.2s)
         Round 8: 72.36% acc, 0.781 loss (33.6s)
      ✅ FedAvg Run 1 Complete: Final=72.36%, Best=72.36% (avg 36.4s/round)
   ✅ FedAvg Summary: 72.36% ± 0.00% (1/1 successful)

🤖 METHOD 2/4: FedProx
   📊 Training 8 rounds × 3 epochs
      🔄 FedProx Run 1 - Training on CUDA
         Round 2: 45.98% acc, 1.418 loss (58.7s)
         Round 4: 63.95% acc, 0.993 loss (60.3s)
         Round 6: 66.47% acc, 0.934 loss (47.3s)
         Round 8: 71.79% acc, 0.795 loss (54.7s)
      ✅ FedProx Run 1 Complete: Final=71.79%, Best=71.79% (avg 57.1s/round)
   ✅ FedProx Summary: 71.79% ± 0.00% (1/1 successful)

🤖 METHOD 3/4: FedNova
   📊 Training 8 rounds × 3 epochs
      🔄 FedNova Run 1 - Training on CUDA
         Round 2: 50.0